In [1]:
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

### Function to convert JSON files to DataFrame structure

In [2]:
def format_json(data):
    init = pd.DataFrame.from_dict(data).loc[0][0]
    for obj in init:
        if type(init[obj])!=list or len(init[obj])!=1:
            init[obj] = [init[obj]]
    df = pd.DataFrame.from_dict(init)
    for i in range(1,pd.DataFrame.from_dict(data).shape[0]):
        curr = pd.DataFrame.from_dict(data).loc[i][0]
        for obj in curr:
            if type(curr[obj])!=list or len(curr[obj])!=1:
                curr[obj] = [curr[obj]]
        df_curr = pd.DataFrame.from_dict(curr)
        df = df.append(df_curr)
    return df

# SURVEY DATA

### Open JSON file, convert to DataFrame

In [3]:
with open("SurveyData.json", "r") as read_file:
    data = json.load(read_file)
df = format_json(data)

### Drop columns missing 80%+ of data

In [4]:
survey_drop = list(df.isnull().sum()[df.isnull().sum(axis=0)>0.8*df.shape[0]].index)
survey = df.copy()
for col in survey_drop:
    print("Dropped column \"" + col +"\"")
survey = survey.drop(survey_drop,axis=1)
print("Remaining columns:")
survey.columns

Dropped column "familyhistory"
Dropped column "otherOrganizationsYouKnow"
Dropped column "numberofIndividualsLivingintheHouse"
Dropped column "numberofChildrenLivingintheHouse"
Dropped column "numberofChildrenLivinginHouseUndertheAgeof5"
Dropped column "yearsLivedinthecommunity"
Dropped column "memberofthefollowingorganizations"
Dropped column "waterAccess"
Dropped column "trashDisposalLocation"
Dropped column "dayMostConvenient"
Dropped column "hourMostConvenient"
Dropped column "yearsLivedinThisHouse"
Dropped column "insuranceNumber"
Dropped column "clinicProvider"
Dropped column "medicalIllnesses2"
Dropped column "whenDiagnosed2"
Dropped column "whatDoctorDoyousee2"
Dropped column "didDoctorRecommend2"
Dropped column "treatment2"
Dropped column "DentalAssessmentandEvaluation"
Dropped column "relationship_id"
Dropped column "picture"
Dropped column "signature"
Remaining columns:


Index(['objectId', 'dob', 'sex', 'marriageStatus', 'occupation',
       'educationLevel', 'telephoneNumber', 'latitude', 'longitude',
       'surveyingUser', 'surveyingOrganization', 'createdAt', 'updatedAt',
       'location', 'communityname', 'city', 'province', 'insuranceProvider',
       'relationship'],
      dtype='object')

### Drop other non-valuable data

In [5]:
survey = survey.drop(['location', 'relationship','surveyingUser','surveyingOrganization','telephoneNumber','communityname'], axis = 1)

  
 
## Cleaning each data column

#### Date of Birth

In [6]:
dobs = []
for date in survey.dob:
    if(date!=None):
        if '-' in str(date):
            temp = str(date).split('-')
            new = temp[1] + "/" + temp[2] + "/" + temp[0]
            dobs.append(new)
        elif '/' in str(date): 
            temp = str(date).split('/')
            new = temp[1] + "/" + temp[0] + "/" + temp[2]
            dobs.append(new)
        else:
            dobs.append(date)
    else:
        dobs.append(None)
survey.dob = dobs

#### Male/Female

In [7]:
survey['M'] = (survey['sex']=='Male').astype(int)
survey['F'] = (survey['sex']=='Female').astype(int)
survey = survey.drop("sex",axis=1)

#### Education Level

In [8]:
survey = survey.replace("lessThanprimary\n","lessThanprimary")

#### Occupation

In [9]:
survey = survey.replace("Ama de casa ","Ama de casa")
survey = survey.replace("Ama d casa","Ama de casa")
survey = survey.replace("Ama d casa ","Ama de casa")
survey = survey.replace("ama de casa ","Ama de casa")
survey = survey.replace("ama de casa","Ama de casa")
survey = survey.replace("Ama de Casa ","Ama de casa")
survey = survey.replace("Ama de Casa","Ama de casa")

survey = survey.replace("Estudiante ","Estudiante")
survey = survey.replace("Student","Estudiante")
survey = survey.replace("Estudia","Estudiante")

survey = survey.replace("Ninguna ","Ninguna")

survey = survey.replace("Agricultura","Agriculture")
survey = survey.replace("Agricultura ","Agriculture")
survey = survey.replace("Agricultor","Agriculture")

survey = survey.replace("Chiripero ","Chiripero")
survey = survey.replace("No trabaja ","No trabaja")
survey = survey.replace("Colmadera","Colmadero")
survey = survey.replace("Jornalero ","Jornalero")
survey = survey.replace("Menor ","Menor")
survey = survey.replace("Comerciante ","Comerciante")

##### Due to the very high variation in occupation, job titles are filtered down to match the 7 most freqeent job titles, otherwise they're labeled as 'other'

In [10]:
work = []
for job in survey['occupation']:
    if job not in list(survey['occupation'].value_counts()[:7].index):
        work.append("Other")
    else: work.append(job)
survey['occupation'] = work

#### Marriage Status

In [11]:
survey = survey.replace("widow\n","widow")

#### Province

In [12]:
prov = []
for p in survey['province']:
    p = str(p)
    if 'sp' in p or 'Sp' in p or 'SP' in p or 'edro' in p or 'aco' in p:
        prov.append("San Pedro de Macoris")
    elif 'vega' in p or 'Vega' in p or (('ga' in p or 've') and ('La' in p or 'la' in p)):
        prov.append("La Vega")
    elif 'onst' in p or "tanza" in p:
        prov.append("Constanza")
    elif 'onsuel' in p:
        prov.append("Consuelo")
    elif 'antiag' in p:
        prov.append("Santiago")
    else: prov.append("None")
pd.Series(prov).value_counts()
survey['province'] = prov

#### City

In [13]:
city = []
survey['city'].value_counts()
for c in survey['city']:
    c = str(c)
    if 'pedro' in c or 'acor' in c or 'edro' in c or 'ro de' in c \
        or 'e mac' in c or 'pm' in c:
        city.append('San Pedro de Macoris')
    elif 'onst' in c or "tanza" in c:
        city.append("Constanza")
    elif 'antia' in c: 
        city.append("Santiago")
    elif 'vega' in c:
        city.append("La Vega")
    elif 'ireo' in c:
        city.append("Tireo")
    else: city.append("None")
pd.Series(city).value_counts()
survey['city'] = city

#### Insurance Provider

In [14]:
ins = []
for i in survey['insuranceProvider']:
    i = str(i)
    if 'Sen' in i or 'sen' in i or 'SeN' in i or 'SEN' in i \
    or 'CeN' in i or 'cen' in i or 'CEN' in i or 'Cen' in i or 'asa' in i:
        ins.append('Senasa')
    elif 'hum' in i or 'ano' in i or 'ARS' in i:
        ins.append('ARS Humano')
    elif 'No' in i or 'nan' in i:
        ins.append("None")
    else:
        ins.append("Other")
pd.Series(ins).value_counts()[:50]
survey['insuranceProvider'] = ins

#### Date of Birth

In [15]:
j = 0
yrs = []
for i in survey['dob']:
    try:
        yr = int(i.split('/')[2])
        if yr>2020 or yr<1915:
            yrs.append(None)
        else:
            yrs.append(2020-yr)
    except:
        yrs.append(None)
pd.Series(yrs).value_counts().sum()
survey['age'] = yrs

### Reindex and send to CSV file

In [16]:
survey = survey.set_index("objectId")

In [17]:
survey.to_csv('survey.csv')

# VITALS DATA

### Open JSON file, convert to DataFrame

In [19]:
with open("Vitals.json", "r") as read_file:
    data = json.load(read_file)
df = format_json(data)

### Drop columns missing 80%+ of data

In [20]:
vitals_drop = list(df.isnull().sum()[df.isnull().sum(axis=0)>0.8*df.shape[0]].index)
vitals = df.copy()
for col in vitals_drop:
    print("Dropped column \"" + col +"\"")
vitals = vitals.drop(vitals_drop,axis=1)
print("Remaining columns:")
vitals.columns

Dropped column "height"
Dropped column "bmi"
Dropped column "temp"
Dropped column "respRate"
Dropped column "bloodOxygen"
Dropped column "painLevels"
Dropped column "hemoglobinLevels"
Remaining columns:


Index(['objectId', 'location', 'weight', 'pulse', 'bloodPressure',
       'bloodSugar', 'client', 'createdAt', 'updatedAt'],
      dtype='object')

### Drop other non-valuable data

In [21]:
vitals = vitals.drop(['location', 'createdAt', 'updatedAt', 'client'], axis = 1)

## Cleaning each data column

#### Blood pressure
##### Splitting into systolic (upper) and diastolic (lower)

In [22]:
bloodPressureUpper = []
bloodPressureLower = []
i=0
j=0
k=0
for bp in vitals['bloodPressure']:
    if(bp!=None):
        bp = str(bp)
        bp = bp.split("/")
        #print(bp)
        if(len(bp)>2):
            bloodPressureUpper.append(None)
            bloodPressureLower.append(None)
            k+=1
        else:
            try:
                upper, lower = int(bp[0]), int(bp[1])
                i+=1
            except:
                upper = 0
                lower = 0
                j+=1
            if upper==0:
                bloodPressureUpper.append(None)
            else:
                bloodPressureUpper.append(upper)
            if lower==0:
                bloodPressureLower.append(None)
            else:
                bloodPressureLower.append(lower)
    else:
        bloodPressureUpper.append(None)
        bloodPressureLower.append(None)
        k+=1
vitals['bpUpper'] = bloodPressureUpper
vitals['bpLower'] = bloodPressureLower
vitals.drop('bloodPressure', axis = 1, inplace = True)

#### Pulse

In [23]:
pulse = []
for p in vitals['pulse']:
    if p!=None:
        try: 
            p = float(p)
            if p>200:
                p = None
        except: 
            p = None
    pulse.append(p)
vitals['pulse'] = pulse

#### Weight

In [24]:
wt = []
for w in vitals['weight']:
    try:
        wt.append(int(w))
    except:
        wt.append(None)
vitals['weight'] = wt

#### Blood Sugar

In [25]:
bs = []
for b in vitals['bloodSugar']:
    try:
        b = int(b)
        if (b>300 or b<20):
            bs.append(None)
        else:
            bs.append(b)
    except:
        bs.append(None)
vitals['bloodSugar'] = bs

### Reindex and send to CSV file

In [26]:
vitals = vitals.set_index("objectId")

In [27]:
vitals.to_csv('vitals.csv')

# ENVIRONMENTAL HISTORY DATA

### Open JSON, convert to DataFrame

In [28]:
with open("HistoryEnvironmentalHealth.json", "r", encoding='utf8') as read_file:
    data = json.load(read_file)
df = format_json(data)

### Drop columns missing 80%+ of data

In [29]:
env_drop = list(df.isnull().sum()[df.isnull().sum(axis=0)>0.8*df.shape[0]].index)
env = df.copy()
for col in env_drop:
    print("Dropped column \"" + col +"\"")
env = env.drop(env_drop,axis=1)
print("Remaining columns:")
env.columns

Remaining columns:


Index(['objectId', 'location', 'yearsLivedinthecommunity',
       'yearsLivedinThisHouse', 'waterAccess', 'typeofWaterdoyoudrink',
       'latrineAccess', 'clinicAccess', 'conditionoFloorinyourhouse',
       'conditionoRoofinyourhouse', 'medicalproblemswheredoyougo',
       'dentalproblemswheredoyougo', 'biggestproblemofcommunity',
       'timesperweektrashcollected', 'wheretrashleftbetweenpickups', 'client',
       'createdAt', 'updatedAt', 'numberofIndividualsLivingintheHouse',
       'numberofChildrenLivinginHouseUndertheAgeof5', 'houseownership',
       'bathroomAccess', 'stoveType'],
      dtype='object')

### Extract Object IDs

In [30]:
pointers = []
for h in env.client.values:
    pointers.append(h['objectId'])
env['objectId'] = pointers

### Drop other non-valuable data

In [31]:
env.drop(["client", "createdAt", "updatedAt"], inplace = True, axis = 1)

## Cleaning each data column

#### Extracting latitude and longitude from Location column

In [32]:
lat = []
lon = []
for h in env.location.values:
    lat.append(h['latitude'])
    lon.append(h['longitude'])
env['latitude'] = lat
env['longitude'] = lon
env = env.drop('location',axis=1)

In [33]:
env.columns

Index(['objectId', 'yearsLivedinthecommunity', 'yearsLivedinThisHouse',
       'waterAccess', 'typeofWaterdoyoudrink', 'latrineAccess', 'clinicAccess',
       'conditionoFloorinyourhouse', 'conditionoRoofinyourhouse',
       'medicalproblemswheredoyougo', 'dentalproblemswheredoyougo',
       'biggestproblemofcommunity', 'timesperweektrashcollected',
       'wheretrashleftbetweenpickups', 'numberofIndividualsLivingintheHouse',
       'numberofChildrenLivinginHouseUndertheAgeof5', 'houseownership',
       'bathroomAccess', 'stoveType', 'latitude', 'longitude'],
      dtype='object')

#### Medical problems where do you go

In [34]:
med_location = []
for m in env['medicalproblemswheredoyougo']:
    m = str(m).lower()
    if 'musa' in m:
        med_location.append("Al Musa Hospital")
    elif 'ramon' in m or 'santana' in m:
        med_location.append("Ramon Santana Hospital")
    elif 'hospital' in m:
        med_location.append("Other Hospital")
    elif 'clinic' in m:
        med_location.append("Clinic")
    else:
        med_location.append("Other")
env['medicalproblemswheredoyougo'] = med_location

In [35]:
env.columns

Index(['objectId', 'yearsLivedinthecommunity', 'yearsLivedinThisHouse',
       'waterAccess', 'typeofWaterdoyoudrink', 'latrineAccess', 'clinicAccess',
       'conditionoFloorinyourhouse', 'conditionoRoofinyourhouse',
       'medicalproblemswheredoyougo', 'dentalproblemswheredoyougo',
       'biggestproblemofcommunity', 'timesperweektrashcollected',
       'wheretrashleftbetweenpickups', 'numberofIndividualsLivingintheHouse',
       'numberofChildrenLivinginHouseUndertheAgeof5', 'houseownership',
       'bathroomAccess', 'stoveType', 'latitude', 'longitude'],
      dtype='object')

#### Dental problems where do you go

In [36]:
dent_location = []
for m in env['dentalproblemswheredoyougo']:
    m = str(m).lower()
    if 'musa' in m:
        dent_location.append("Al Musa Hospital")
    elif 'ramon' in m or 'santana' in m:
        dent_location.append("Ramon Santana Hospital")
    elif 'hospital' in m:
        dent_location.append("Other Hospital")
    elif 'dentist' in m or 'dentista' in m:
        dent_location.append("Dentist")
    elif 'clinic' in m: 
        dent_location.append("Clinic")
    else:
        dent_location.append("Other")
env['dentalproblemswheredoyougo'] = dent_location

#### Biggest problem of community

In [37]:
comm_prob = []
for m in env['biggestproblemofcommunity']:
    m = str(m).lower()
    if 'agua' in m:
        comm_prob.append("Water")
    elif 'basura' in m:
        comm_prob.append("Garbage")
    elif 'salud' in m:
        comm_prob.append("Health")
    elif 'calle' in m or 'carre' in m:
        comm_prob.append("Roads")
    elif 'empleo' in m:
        comm_prob.append("Unemployment")
    elif 'delinc' in m:
        comm_prob.append("Delinquents")
    elif 'luz' in m:
        comm_prob.append("Lighting")
    else:
        comm_prob.append("Other")
env['biggestproblemofcommunity'] = comm_prob

#### Times per week trash is collected

In [38]:
trash = []
for m in env['timesperweektrashcollected']:
    m = str(m).lower()
    try: 
        trash.append(int(m))
    except: 
        if '1' in m or 'una' in m or 'semanal' in m:
            trash.append(1)
        elif '2' in m:
            trash.append(2)
        elif '3' in m:
            trash.append(3)
        elif '4' in m:
            trash.append(4)
        elif '5' in m:
            trash.append(5)
        elif '6' in m:
            trash.append(6)
        elif '7' in m:
            trash.append(7)
        elif 'no' in m or 'none' in m or 'nunca' in m:
            trash.append(0)
        elif 'guema' in m or 'quema' in m:
            trash.append("Burned")
        else: 
            trash.append("Unknown")
env['timesperweektrashcollected'] = trash

#### Where trash is left between pickups

In [39]:
trash_left = []
for t in env['wheretrashleftbetweenpickups']:
    t = str(t).lower()
    if 'calle' in t:
        trash_left.append("street")
    elif 'frent' in t or 'fuer' in t or 'patio' in t:
        trash_left.append("outside house")
    elif 'quem' in t or 'guem' in t: 
        trash_left.append("burned")
    elif 'cer' in t or 'sera' in t:
        trash_left.append("sidewalk")
    else: 
        trash_left.append("other")
pd.Series(trash_left).value_counts()[:30]
env['wheretrashleftbetweenpickups'] = trash_left

In [40]:
env['objectId']

0    Psto9poSCx
0    y6DJO1kLFl
0    sdPbSYrGCW
0    GiArohoFUo
0    rnNw8AxEix
        ...    
0    uKN9cHxFDl
0    pmE94lhtoq
0    Tn1ljw7wVy
0    Tn1ljw7wVy
0    QpmL0Z6uFm
Name: objectId, Length: 7546, dtype: object

#### Number of individuals living in the house

In [41]:
household=[]
for n in env['numberofIndividualsLivingintheHouse']:
    n = str(n).lower()
    try:
        household.append(int(n))
    except:
        try: 
            household.append(int(n[:2]))
        except:
            try:
                household.append(int(n[:1]))
            except:
                if 'una' in n or 'uno' in n:
                    household.append(1)
                elif 'dos' in n:
                    household.append(2)
                elif 'tres' in n:
                    household.append(3)
                elif 'cuatro' in n:
                    household.append(4)
                elif 'nan' in n or 'none' in n:
                    household.append(4) #manually calculated median
                else: 
                    household.append(4) #manually calculated median
env['numberofIndividualsLivingintheHouse'] = household

#### Number of children living in the house under the age of 5

In [42]:
kids = []
for n in env['numberofChildrenLivinginHouseUndertheAgeof5']:
    n = str(n).lower()
    try:
        kids.append(int(n))
    except:
        try: 
            kids.append(int(n[:2]))
        except:
            try:
                kids.append(int(n[:1]))
            except:
                if 'una' in n or 'uno' in n:
                    kids.append(1)
                elif 'dos' in n:
                    kids.append(2)
                elif 'tres' in n:
                    kids.append(3)
                elif 'cuatro' in n:
                    kids.append(4)
                elif 'nan' in n or 'none' in n:
                    kids.append(0)
                else: 
                    kids.append(0)
pd.Series(kids).value_counts()
env['numberofChildrenLivinginHouseUndertheAgeof5'] = kids

### Reindex and send to CSV file

In [43]:
env = env.set_index("objectId")

In [44]:
env.to_csv("history_environment.csv")

# MEDICAL EVALUATION DATA

### Open JSON, convert to DataFrame

In [45]:
with open("EvaluationMedical.json", "r", encoding='utf8') as read_file:
    data = json.load(read_file)
df = format_json(data)

### Drop columns missing 80%+ of data

In [46]:
med_drop = list(df.isnull().sum()[df.isnull().sum(axis=0)>0.8*df.shape[0]].index)
med = df.copy()
for col in med_drop:
    print("Dropped column \"" + col +"\"")
med = med.drop(med_drop,axis=1)
print("Remaining columns:")
med.columns

Dropped column "abnormal_bleeding"
Dropped column "difficulty_breathing"
Dropped column "mental_issues"
Dropped column "description"
Dropped column "diagnosis"
Dropped column "suggested_treatment"
Dropped column "received_treatment"
Dropped column "needsAssessmentandEvaluation"
Dropped column "chronic_condition_hypertension"
Dropped column "chronic_condition_diabetes"
Dropped column "chronic_condition_other"
Dropped column "pain"
Dropped column "AssessmentandEvaluation_Surgical_Guess"
Remaining columns:


Index(['objectId', 'location', 'AssessmentandEvaluation', 'planOfAction',
       'notes', 'surveyingUser', 'surveyingOrganization', 'client',
       'createdAt', 'updatedAt', 'duration', 'seen_doctor',
       'received_treatment_description', 'received_treatment_notes',
       'part_of_body', 'trauma_induced', 'condition_progression',
       'AssessmentandEvaluation_Surgical', 'immediate_follow_up',
       'part_of_body_description'],
      dtype='object')

### Extract Object IDs

In [47]:
client = []
for c in med['client']:
    client.append(c['objectId'])
med['objectId'] = client

### Drop other non-valuable data

In [48]:
med = med.drop(["client", "createdAt", "updatedAt", "notes", "surveyingUser", "surveyingOrganization", 
                "received_treatment_notes", "part_of_body_description"], axis = 1)

## Cleaning each data column

#### Extracting latitude and longitude from Location clumn

In [49]:
lat = []
lon = []
for h in med.location.values:
    lat.append(h['latitude'])
    lon.append(h['longitude'])
med['lat'] = lat
med['lon'] = lon
med = med.drop('location', axis = 1)

#### Assessment and evaluation

In [50]:
medeval = []
for m in med['AssessmentandEvaluation']:
    try:
        if 'No' in m:
            medeval.append("No")
        else: medeval.append("Yes")
    except:
        medeval.append("No")
med['AssessmentandEvaluation'] = medeval

#### Plan of action

In [51]:
poa = []
for p in med['planOfAction']:
    p = str(p).lower()
    if 'orto' in p:
        poa.append("orthopedics")
    elif ('general' in p  or 'genral' in p) and 'med' in p:
        poa.append("general medicine")
    elif 'cardio' in p:
        poa.append("cardiology")
    elif ('general' in p or 'genral' in p) and 'cirug' in p:
        poa.append("general surgery")
    elif 'urol' in p:
        poa.append("urology")
    elif 'neur' in p:
        poa.append("neurology")
    elif 'derma' in p:
        poa.append("dermatology")
    elif 'ginec' in p:
        poa.append("gynecology")
    elif 'ofta' in p or 'opta' in p:
        poa.append("ophthalmology")
    elif 'otor' in p:
        poa.append("laryngology")
    elif 'gastro' in p:
        poa.append("gastroenterology")
    elif 'pedia' in p:
        poa.append("pediatrics")
    elif ('no' in p and 'neces' in p) or 'none' in p:
        poa.append("none")
    else: poa.append("other")
med['planOfAction'] = poa

#### Duration since last seen doctor

In [52]:
med['duration'] = med['duration'].replace('Bdjajbd',None)

#### Part of body

In [53]:
med['part_of_body'] = med['part_of_body'].replace("",None)

### Reindex and send to CSV file

In [54]:
med = med.set_index("objectId")

In [55]:
med.to_csv("evaluation_medical.csv")